# 1. Three number sum

## 1.1. Brute Force Solution

The simpliest solution is the three nested for loops which costs $O(n^3)$ time.  Notice the looping ranges, at the final iteration, the value of the indices will be $i=n-3$ , $j=n-2$ and $k=n-1$. as the Python `range(x,y)` command ranges $[x: y-1]$ therefore,
* for $i : y-1 = n-3 \rightarrow y=n-2$  
* for $j : y-1 = n-2 \rightarrow y=n-1$
* for $k : y-1 = n-1 \rightarrow y=n$    

In [2]:
# three num sum brute force
def three_sum_bf(arr, target):
    ret = []
    n = len(arr)
    # at the last iteration i=n-3, j=n-2 and k=n-1
    for i in range(n-2):
        for j in range(i+1, n-1):
            for k in range(j+1, n):
                if arr[i]+arr[j]+arr[k] == target:
                    ret.append([ arr[i],arr[j],arr[k] ] )
    return ret

In [3]:
arr = [12,3,1,2,-6,5,-8,6]
target =0 
result = three_sum_bf(arr, target)
print(result)

[[3, 5, -8], [1, -6, 5], [2, -8, 6]]


## 1.2. Optimal Soluition

The optimal solution is a bit non-trivial. The core logic is as follows,
1. if $a_i + a_j + a_k = t$ then $t - a_i = a_j + a_k$ which could be assumed as $r_i = s_{i,j}$, where $r_i$ is the remainder w.r.t. $a_i$ and $s_{i,j}$ is the sum. 
2. If $A$ is sorted then the $(a_j,a_k)$ pair can be found in linear time based on the follwoing conditions..
$$\begin{equation}
  r_i =
    \begin{cases}
      \lt s_{ij} & \text{decrese $s_{i,j}$ using moving $j$ to left }\\
      \gt s_{ij} & \text{increse $s_{i,j}$ using moving $i$ to right }\\
      = s_{ij} & \text{match found, append $[a_i,a_j,a_k]$ and move both $i$ and $j$}
    \end{cases}       
\end{equation}$$
3. the $a_i$ will scan the array in $O(n)$ time. For each iteration the remiander array needs to be scanned in $O(n)$ time, as each time either or both the pointer moves and in liniear time they'll meet which will break the inner loop.

__Complexity analyis__

| Task | Cost |
|---|---|
| Sorting | $O(nlog(n))$ |
| Outer loop | $O(n)$ |
|Inner loop | $O(n)$ |
|Total | $T(n) = O(nlogn + n^2) = O(n^2)$ <br> $S(n) = O(n)$|


In [4]:
# three number sum optimal solutions
def three_sum(arr, target):
    arr = sorted(arr)   # sort the array
    n = len(arr)
    ret = []
    for i in range(n-2):
        remainder = target - arr[i]    # calculate r_i
        # set window 
        j = i+1
        k = n-1 
        while j < k:    # windowing logic
            part_sum = arr[j] + arr[k]  # calculate s_ij
            if remainder < part_sum:
                j-=1
            elif remainder > part_sum:
                i+=1
            else:      #match found 
                ret.append( [arr[i], arr[j], arr[k] ])
                i+=1
                j-=1
    return ret

In [5]:
arr = [12,3,1,2,-6,5,-8,6]
target =0 
result = three_sum_bf(arr, target)
print(result)

[[3, 5, -8], [1, -6, 5], [2, -8, 6]]


# 2. Smallest Differrece 

Given two array $A$, $B$ of iniitegers (perhaps different sizes), find the pair $a_i \in , b_j \in B$, such that, $|a_i - b_j|$ is minimum. 

## 2.1. Brute Force Solution
The Brute force solution is straight forward. 
1. Search all pairs of numbers $(a_i,b_j)$ from $A$ and $B$ 
2. Compare $|a_i - b_j|$ with a presumed min and maintain the min pair
3. Return the min pair

However, the solution is an quadratic time, $O(n^2)$ solution. 

In [8]:
def smallest_diff_bf(arr_1, arr_2):
    global_diff = None
    ret = [None]*2
    for i in range(len(arr_1)):
        for j in range(len(arr_2)):
            local_diff = abs(arr_1[i] - arr_2[j]) 
            if global_diff == None or (global_diff is not None and local_diff < global_diff):
                global_diff = local_diff
                ret = [arr_1[i], arr_2[j]]
    return ret


In [9]:
arr_1 = [-1,5,10,20,28,3]
arr_2 = [26,134,135,15,17]
smallest_diff_bf(arr_1, arr_2)

[28, 26]

## 2.2. Optimal Solution 
There are two optimal solutions for these, that can solve it in $T(n) = O(nlogn) , S(n)=O(1)$

### 2.2.1. Modefied Bineary Search
* Here we use a modefied binary search logic to find the closest number to each item from $A$ in $B$. 
* Each search takes $O(\log(|B|)))$ time for $|A|$ elements and $O(m\log(m))$ for sorting B to perform binary search, i.e., $n\log(m) + m\log(m) = O(n\log(n))$ 
* The modification is, if the target is found it's returned as usual, otherwise the closest element is returned. 

In [12]:
def find_closest(arr, target, l, h):    # modefied binary search (heelper function)
    m = (l+h) //2 # find mid position 
    if arr[m] == target:
        return target
    # mod starts here
    elif (h-l) == 1:
        if abs(target - arr[l]) < abs(target - arr[h]):   #return closest 
            return arr[l]
        else:
            return arr[h]
    # mod ends here 
    elif target < arr[m]:    # target on the left
        return find_closest(arr, target, l, m)
    elif target > arr[m]:    # target on the right
        return find_closest(arr, target, m, h)
    else:
        pass

def smallest_diff(arr_1, arr_2):
    arr_2 = sorted(arr_2)  # O(nlogn)
    global_min = None
    ret = [None]*2

    for item_1 in arr_1:    # O(n)
        item_2 = find_closest(arr=arr_2, target=item_1, l=0, h=len(arr_2)) # O(logn)
        local_min = abs(item_2 - item_1)
        if global_min == None or (global_min is not None and global_min > local_min):
            global_min = local_min
            ret = [item_1, item_2]
    return ret

In [11]:
arr_1 = [-1,5,10,20,28,3]
arr_2 = [26,134,135,15,17]
smallest_diff(arr_1, arr_2)

[28, 26]

# 3. Selective Shift 

Given an array on integers $A$ and a value $v$, write a program that shifts all the occurance of $v$ to left in-place.

## 3.1. Buteforce: (pop and append)

A simplistic approach is to scan through the array and remove elements when $v$ is encountered and append it at the end. However, it has two issues.
1. It's not a in-place tranformation 
2. Mutating lists is time consuming as each delete opeation casuses a series of left-shifts of the RHS elements.   

In [35]:
def shift_right_bf(arr, val):
    current_index = 0
    limit = len(arr)
    while current_index < limit:
        print(f'ci={current_index}, lim={limit}, arr={arr}')
        if arr[current_index] == val:
            arr.append(arr.pop(current_index)) # changes the index, hard to maintain
            limit-=1
        else: 
            current_index+=1
    return arr

In [36]:
arr = [2,1,2,2,2,3,4,2]
val = 2
shift_right_bf(arr, val)

ci=0, lim=8, arr=[2, 1, 2, 2, 2, 3, 4, 2]
ci=0, lim=7, arr=[1, 2, 2, 2, 3, 4, 2, 2]
ci=1, lim=7, arr=[1, 2, 2, 2, 3, 4, 2, 2]
ci=1, lim=6, arr=[1, 2, 2, 3, 4, 2, 2, 2]
ci=1, lim=5, arr=[1, 2, 3, 4, 2, 2, 2, 2]
ci=1, lim=4, arr=[1, 3, 4, 2, 2, 2, 2, 2]
ci=2, lim=4, arr=[1, 3, 4, 2, 2, 2, 2, 2]
ci=3, lim=4, arr=[1, 3, 4, 2, 2, 2, 2, 2]


[1, 3, 4, 2, 2, 2, 2, 2]

## 3.2. Optimal Solutins

The optimal solution works on the follwing logic. If the we take two poiners $i$ and $j$ initialized at the left and the right extremes of $A$, and follows the primitive windowing logic that $i\lt j$. Then, there could be four possible occurances

| i | j | Situation | operation |
|--|--|--|--|
| $=v$ | $=v$ | $i$ wants to move but $j$ can't have it | look for next $j$ ($j--$) |
| $=v$ | $\neq v$ | $i$ wants to move but $j$ can have it | swap $a_i,a_j$ then shift both $i$ and $j$|
| $\neq v$ | $=v$ | $i$ doesn't want to move but $j$ can't have it | look for next $i$ and $j$ ($i++ , j--$) |
| $\neq v$ | $\neq v$ | $i$ doesm't want to move but $j$ can have it | look for next $i$ ($j++$) |


In [28]:
def shift_right(arr, val):
    i = 0
    j = len(arr)-1
    while i < j:
        print(arr)
        if arr[i] == val:   # i wants to send
            if arr[j] == val:   # j doesn't want to receive 
                j-=1
            else:               # j wants to receive             
                arr[i],arr[j] = arr[j], arr[i]
                i+=1
                j-=1  
        else:               # i doesn't want to send
            if arr[j] == val:   # j doesn't want to receive 
                i+=1
                j-=1
            else:               # j wants to receive  
                i+=1
    return arr

In [29]:
arr = [2,1,2,2,2,3,4,2]
val = 2
shift_right(arr, val)

[2, 1, 2, 2, 2, 3, 4, 2]
[2, 1, 2, 2, 2, 3, 4, 2]
[4, 1, 2, 2, 2, 3, 2, 2]
[4, 1, 2, 2, 2, 3, 2, 2]
[4, 1, 3, 2, 2, 2, 2, 2]


[4, 1, 3, 2, 2, 2, 2, 2]

# 4. Monotonic Array

An array $A$ is said to be monotonic if $i \lt j \implies a_i \le a_j$ where $i,j$ are the indices and $a_i,a_j \in A$. Given an array $A$ determine if $A$ is monotonic or not, i.e., $f:A \rightarrow \{ T,F\}$

__Points to remember__
1. A monotonic array perhaps not be a _Strictly Monotonic_ Array.
2. A repeating array e.g. `[1,1,1,1]` is also monotonic but not strictly monotonic.
3. A monotonic array could be either monotonic increasing or decreasing but not both ot a flat one. 

__Logic__
1. Scan for each consicutive elements from left to right. there could be three conditions
    * $a_i \lt a_{i+1}$ : monotonic decresing, all next pairs should be mono. dec.
    * $a_i \gt a_{i+1}$ : monotonic increasing, all next pairs should be mono. inc. 
    * $a_i = a_{i+1}$ : flat, look for next pair 
2. In each case if a pair violates the pattern, return False 
3. Return true at the end  


In [37]:
def isMonotonic(array):
    # Write your code here.
	n = len(array)
	i=0
	while i < n-1:   
		j = i+1
		if array[i] == array[j]: # not dicidable 
			i+=1
		elif array[i] < array[j]:  # mono inc.
			while j < n:
				if array[i] > array[j]: # mono inc violation 
					return False
				i+=1 ; j+=1
		elif array[i] > array[j]:  # mono dec.
			while j < n:
				if array[i] < array[j]: # mono dec violation 
					return False
				i+=1 ; j+=1
		else:
			pass
	return True

In [38]:
arr = [-1, -5 , -10 , -1100, -1100, -1101, -1102, -9001]  
isMonotonic(arr)

True

# 5. Longest Peak
 
 A peak is defined as adjacenct integers in an array that are __strictly__ monotonic increasing upto a point and __strictly__ decreaing affterwards. The length of the peack is the sum of the length of its LHS and RHS. <br>
 Given an array of integer return the length of the longest peack. 

__Logic__
1. The first and land items i.e., $a_0$ and $a_{n-1}$ can't be a peack, as they don't have both the sides. 
2. Find a value within $A[1:(n-2)]$ which is a peak, i.e. $a_{i-1} \lt a_i \lt a_{i+1}$
3. Calculate the length of the strictly monotonic decresing LHS i.e. $L = A[i-l : i-1]$ for $a_i$ , and the same for the RHS i.e., $R = A[i+1 : i+r]$
4. Calculate the run length of the peak $|L|+|R|+1$ and compare with a presumed $max$ 
5. Return the $max$   

In [39]:
def longestPeak(array):
    # Write your code here.
	n = len(array)
	global_len = None
	for i in range(1,n-1):
		if array[i-1] < array[i] and array[i] > array[i+1]:   # peak check passed
			l=r=i # initialize left and right pointer 
			# left scanning for mono dec len 
			while l > 0 and array[l-1] < array[l]:  # mono dec condition is valid
				l-=1
			# right scanning for mono dec len 
			while r < n-1  and  array[r+1] < array[r]:  # mono dec condition is valid 
				r+=1
			local_len = (r-l+1)
			if global_len == None or (global_len is not None and global_len < local_len):
				global_len = local_len
	if global_len == None:
		return 0
	else:
		return global_len

In [40]:
arr = [1, 2, 3, 3, 4, 0, 10, 6, 5, -1, -3, 2, 3]
longestPeak(arr)

6

# 6. Array of Product

Given a array $A = \{a_i\}$ produce a new array $B = \{b_i | b_i = \prod_{j=0 , j\neq i}a_j\}$ i.e. the elements of $B$ is the product of elements of $A$ except the corresponding element. <br>
Perform the problem without using division. 

## 6.1. Bruteforce solution: calculate product and keep deviding
The obvious solution to this problem is to calculate the product $p$ of the array $A$ and $b_i = \frac{p}{a_i}$.

In [43]:
def arrayOfProducts_bf(array):
    # Write your code here.
	p = 1
	for i in array:
		p*=i
	return [ int(p/i) for i in array ]

In [44]:
arr = [5, 1, 4, 2]
arrayOfProducts_bf(arr)

[8, 40, 10, 20]

However, the problem asked to accomplish the task without using division, because this solution will not work if there are multiple occurances of Zeros. 

In [46]:
arr = [0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
try:
    print(arrayOfProducts_bf(arr))
except:
    print('error')


error


# 6.2. Optimal Solution

The optimal solution is not primitve, it works as follows. 

1. Calculate two temprary arrys $L$ and $R$ such that, 
    * $L$ contains the products of the elements of $A$ from left to right, i.e. $L = \bigg  \{l_{i+1} = l_{i}\times a_i | l_0=1\bigg\}$ and 
    * $R$ contains from right to left $R = \bigg\{ r_{j-1} = r_{j}\times a_j | r_{n-1}=1 \bigg\}$.
2. element-wise product of $L$ and $R$ is $B$, i.e., $B=\bigg\{ b_i = l_i\times r_i | l_i\in L , r_i \in R \bigg\}$

In [47]:
def arrayOfProducts(array):
    # Write your code here.
	n = len(array)
	
	L = [None]*n
	R = [None]*n
	
	lp = rp = 1
	
	for i in range(n):
		L[i] = lp
		R[n-i-1] = rp
		
		lp *= array[i]
		rp *= array[n-i-1]
		
	return [ L[i]*R[i] for i in range(n) ]

In [48]:
arr = [5, 1, 4, 2]
print(arrayOfProducts(arr))

arr = [0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print(arrayOfProducts(arr))

[8, 40, 10, 20]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [49]:
# 7. First Duplicate 

def firstDuplicateValue(array):
    # Write your code here.
	for item in array:
		idx = abs(item)
		if array[idx -1 ] < 0:
			return idx
		array[idx -1] *= -1
	return -1